# Naive Bayes

**Naive Bayes Algorithm:**

输入:训练数据$T=\{(x_1,y_1),(x_2,y_2),...,(x_N,y_N)\}$,其中$x_i=(x_i^{1},x_i^{2},...,x_i^{n})^{T}$,$x_{i}^{j}$是第$i$个样本的第$j$个特征,$x_{i}^{j} \in{a_{j1},a_{j2},...,a_{jl}}$,$a_{jl}$是特征可能的取值;$y_i\in \{c_1,...,c_N\}$;实例$x$

输出:实例$x$的分类

(1) 计算先验概率以及条件概率:


$P(Y=c_k)=\frac{\sum_{i=1}^{N}I(y_i=c_k)}{N},k=1,2,...,N$

$P(X^{(j)}=a_{jl}|Y=c_{k})=\frac{\sum_{i=1}^{N}I(x_{i}^{j}=a_{jl},y_i=c_k)}{\sum_{i=1}^{N}I(y_i=c_k)}$

(2) 计算后验概率即预测结果:

$P(Y=c_k) \prod_{j}P(X^{(j)}=a_{jl}|Y=c_{k})$

(3) 确定实例:

$C=f(x)=\underset{c_k}{argmax}\;\frac{P(Y=c_k) \prod_{j} P(X^{(j)}=x^{(j)}|Y=c_k)}{\sum_{n=1}^{N}P(Y=c_k) \prod_{j} P(X^{(j)}=x^{(j)}|Y=c_k)}$

但是由于我们是选则最大的$c_k$所以我们没有必要计算分母部分,所以最终的形式为:

$C=f(x)=\underset{c_k}{argmax}\;P(Y=c_k) \prod_{j} P(X^{(j)}=x^{(j)}|Y=c_k)$

**Ps:**

有些情况下:如果条件概率接近0(一般情况下不会),这会对后验概率产生影响(因为是$\prod$),所以我们需要在条件概率中加入一个极小的值$\epsilon $防止概率为0.则条件概率变为:

$P(X^{(j)}=a_{jl}|Y=c_{k})=\frac{\sum_{i=1}^{N}I(x_{i}^{j}=a_{jl},y_i=c_k)+\epsilon}{\sum_{i=1}^{N}I(y_i=c_k)+\epsilon}$


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1 Multinomial Naive Bayes

尝试使用表4.1的训练数据学习一个Naive Bayes分类器并确定$x=(2,S)^{T}$的类标签.在表中$X^{(1)},X^{(2)}$为特征,$Y$为类标签.

![](picture/52.png)

### 1.1 Load Data

在计算的时候,我们需要使用[pandas](https://pandas.pydata.org/)来处理数据会更加方便.

对于pandas有几个比较重要的点:

- np.nan能参与到计算中,但计算的结果总是NaN.
- None是Python自带的,其类型为python object.因此,None不能参与到任何计算中,pandas会将其转换NaN.
- DataFrame:可以看成是2D的表格

**Ps:**

在Numpy中,如果多维数组中含有多中类型,则等级是str > float > int.

In [58]:
def LoadData_():
    """
    Load data set 
    
    Return:
    ------
        data: DataFrame.
        
    Note:
    ----
        last column: labels
    """
    datasets = np.array([[1,1,1,1,1,2,2,2,2,2,3,3,3,3,3],
                  ['S','M','M','S','S','S','M','M','L','L','L','M','M','L','L'],
                  [-1,-1,1,1,-1,-1,-1,1,1,1,1,1,1,1,-1]]).T
    columns = np.array(['X1','X2','Y'])

    data = pd.DataFrame(data=datasets,columns=columns)
    return data

In [59]:
data = LoadData_()

### 1.2 Build Naive Bayes

在构建Naive Bayes之前,我们要先整理下逻辑:

(1) 我们需要先获取labels中有多少种类别,比如例子中是两个.这样我们才能继续计算概率值.

(2) 先计算先验概率.在计算完毕之后我们需要将其保留,所以我们应该先设置一个保留先验概率的矩阵,其形状应该是(K,).

```python
 pri_prob= np.zeros((len(set_labels_list)))```
 
(3) 计算条件概率的时候应该是两层循环:外层应该是label,内层是特征,这样才能计算出每一类label下不同特征的概率.

```python

for j in range(len(set_labels_list)):
       ...
        for k in range(len(test_X)):
            .... ```
            
(4) 将条件概率全部计算出来了之后,需要保留在一个数组内,方便最后比较,所以我们要建立一个预测值保留的数组,其形状应该是(m,k),m是预测样本的个数,k是labels的种类数.

```python

cache_predict = np.zeros((len(set_labels_list))) ```

In [60]:
def NaiveBayes(data,test_X,gamma,is_print=False):
    """
    Build Naive Bayes
    
    Parameters:
    ----------
        data: training set.
        test_X: testing set.
        gamma: Laplace smoothing
        is_print: is print label category and pos posterior probability.
    """
    
    # 获取labels
    labels = data.iloc[:,-1]
    # 获取labels的类别个数K
    labels_ = np.array(list(set(labels)))
    len_labels_ = labels_.shape[0]
    
    # 先验概率
    pri_prob= np.zeros((len_labels_))
    
    
    # 预测值保留数组
    m,n = test_X.shape
    cache_predict = np.zeros((m,len_labels_))
    
    # 计算先验概率
    for i in range(len_labels_):
        P_y = (labels[labels == labels_[i]].size + gamma) / (labels.size + len_labels_ * gamma)
        pri_prob[i] = P_y
    
    # 计算条件概率
    for i in range(m):
        for j in range(len_labels_):
            Conditional_Prob = 1
            for k in range(n):
        
                data_label = data[labels==labels_[j]] # 该标签下的所有数据
                future_k = data_label.iloc[:,k]  # test_X 的第k个特征列
                molecule = data_label[future_k == test_X[i,k]].shape[0] + gamma # 分子部分
                Sj = len(set(future_k))
                denominator = data_label.shape[0] + (Sj * gamma) # 分母部分
                
                Conditional_Prob *= molecule /denominator # 计算条件概率
                
            Pos_proba = pri_prob[j] * Conditional_Prob # 计算后验概率
            
            cache_predict[i,j] += Pos_proba
        
    # if ture,print labels_ and predict probability array.
    if is_print:
        print(labels_)
        print(cache_predict)
        
    best_predict_index = np.argmax(cache_predict,axis=1)
    return labels_[best_predict_index]

In [61]:
test_x = np.array([['2','S']])
predic_label = NaiveBayes(data=data,test_X=test_x,gamma=1,is_print=True)
print('Predict label is: ',predic_label)

['1' '-1']
[[0.03267974 0.06100218]]
Predict label is:  ['-1']


### 2 文本分类案例

尝试将一段文本进行分类处理,可用于垃圾邮件分类,恶意评论,恶意发言等在NLP方面的应用

对于文本处理,我们通常会将文本处理成"Hot"形式(Word to vector).

例如:

- 文本(1):A,B,B,C,D
- 文本(2):E,D,G,H

那么“Hot”的形式就是将所有文本融合,对应的索引位置为1,这样就可以实现从某类不可计算的"东西"转换成数字.

比如融合(去重)的形式为:

A,B,C,D,E,G,H

词集模型为(set of words model):

以"Hot"来表示文本:

文本(1)向量为:[1,1,1,1,0,0,0]

文本(2)向量为:[0,0,0,1,1,1,1]

**Ps:**

到目前未知，我们将每个词的出现与否作为一个特征，这可以被描述为词集模型(set-of-words model)。如果一个词在文档中出现不止一次，这可能意味着包含该词是否出现在文档中所不能表达的某种信息，这种方法被称为词袋模型(bags-of-words model)。在词袋中，每个单词可以出现多次，而在词集中，每个词只能出现一次。为了适应词袋模型，每当遇到一个单词时，它会增加词向量中的对应值，而不只是将对应的数值设为1。

另外，词汇表中的一小部分单词却占据了所有文本用词的大部分。产生这种现象的原因是因为语言中大部分都是冗余和结构辅助性内容。一个常用的方法是移除高频词，另一个常用的方法是不仅移除高频词，同时从某个预定词表中移除结构上的辅助词。该词表称为停用词表（stop word list），当然也可以花大量时间对切分器进行优化。


词袋模型(bag of words model):

以"Hot"来表示文本:

文本(1)向量为:[1,2,1,1,0,0,0]

文本(2)向量为:[0,0,0,1,1,1,1]

#### 2.1加载数据集

In [6]:
def loadDataSet():
    """
    Create dataset
    
    Returns:
        posting list and classVec
    """
    postingList = [['my','dog','has','flea','problems','help','please'],
                  ['maybe','not','take','him','to','dog','park','stupid'],
                  ['my','dalmation','is','so','cute','I','love','him'],
                  ['stop','posting','stupid','worthless','grabage'],
                  ['mr','licks','ate','my','steak','how','to','stop','him'],
                  ['quit','buying','worthless','dog','food','stupid']]
    classVec = np.array([0,1,0,1,0,1]) # 1 is absive,0 not
    
    return postingList,classVec

In [7]:
postingList,classVec = loadDataSet()

#### 2.2 build Vocabulary

首先我们需要创建词汇表(Vocabulary):

将所有给定的样本进行去重并保存在一个数组或者列表中.

**Ps:**

随着训练样本的增多,这个词汇表会越来越大,由词汇表组成的"Hot"向量也会越来越大.

In [8]:
def createMixWord(postingList):
    """
    create vocabulary.
    
    Parameters:
    ----------
        postingList:training set.
    Return:
    ------
        list(mix_word):Vocabulary
    """
    mix_word = set([])
    for line in postingList:
            mix_word = mix_word | set(line)
            
    return list(mix_word)
    

In [18]:
mix_word = createMixWord(postingList)

### 2.3 Word to Vector

将训练样本转换成"Hot"的形式,同样训练样本也需要转换成"Hot"的形式,如果训练样本中的word没有在词汇表中,那么对应单词只能抛弃.所以可以看出词汇表的大小还是非常重要的.

为了将所有训练数据的"Hot"向量放在一起方便接下去计算,我们需要先初始化最后的Vec_words,用来存放所有训练样本的"Hot"向量.
其形状是:(m,n),m:训练样本数量,n:训练样本所有的可能特征.

```python
Vec_words = np.zeros((m,n))```




In [47]:
def Word2Vec_set(mix_word,postingList):
    """
    Implementation word to vector.
    
    Parameters:
    -----------
        mix_word: vocabulary.
        postingList: training set.
    Return:
    ------
        Vec_words: all "Hot" at training set.
    """
    
    m,n = len(postingList),len(mix_word)
    
    Vec_words = np.zeros((m,n))
    
    for i in range(m):
        for word in postingList[i]:
            try:
                index_ = mix_word.index(word)
                Vec_words[i,index_] = 1
            except:
                print('the word {} is not in my Vocabulary!'.format(word))
            
            
    return Vec_words

In [48]:
Vec_words = Word2Vec_set(mix_word,postingList)

来查看一下"Hot"的训练样本

In [49]:
display(Vec_words)

array([[1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
        0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0.]])

以第一个样本为例,在Vec_words[0]\[0\]为1代表着词集中的第一个单词在第一个样本中.Vec_words[0]\[1\]为0则表示词集中的第一个单词不在第一个样本中

In [50]:
mix_word[0] in postingList[0]

True

#### 2.4 Build word of Naive Bayes

在构建Naive Bayes的过程中,先计算先验概率后计算条件概率,最后得到后验概率.

(1)计算先验概率:

计算先验概率的之前,也是也需要创建一个保存先验概率的数组的.其形状为(K,)因为我们有K个类别需要分.因为是对于二分类且我们的labels是基于0,1的所以直接计算$labels=1$的概率即可得到$labels=0$的概率

```python
pri_prob= np.zeros(K)
....
P_y1 = classVec.sum() / classVec.shape[0]
P_y0 = 1 - P_y1```

(2) 计算条件概率:

在计算条件概率之前,我们需要先将测试样本转换为向量的形式,然后再计算条件概率.

因为我们这里选用的是词集模型,所以在对比测试样本和训练样本有多少个相同的时候只要使两个样本相加,得到的结果大于1的即视为在训练样本中出现:

也就是说,在求条件概率$\sum_{i=1}^{N}I(x_{i}^{j}=a_{jl},y_i=c_k)$

```python

vec_test_words = Word2Vec_set(mix_word,test_words)
...
same_words = np.where((vec_test_words + label_data)>1)[0]
molecule = same_words.shape[0] + gamma 

```

(3)计算后验概率,比较两类后验概率的大小,选择最大的作为预测标签.

In [56]:
def NaiveByes_word(mix_word,Vec_words,test_words,classVec,gamma,is_print=False):
    """
    Build word of Naive Bayes.
    
    Parameters:
    ----------
        mix_word: vocabulary
        Vec_words: training set.
        test_words: testing set.
        classVec: labels
        gamma: Laplace smoothing
        is_print: is print label category and pos posterior probability.
    
    Return:
    ------
       predict_label: Predict label. 
    
    """
    K = len(set(classVec))
    list_set_classVec = list(set(classVec))
    m = len(test_words)
    
    # 初始化先验概率数组
    pri_prob= np.zeros(K)
    # 初始化后验概率
    Pos_proba = np.zeros((m,K))
    # 计算先验概率,对于二分类
    P_y1 = classVec.sum() / classVec.shape[0]
    P_y0 = 1 - P_y1
    
    # 计算条件概率
    vec_test_words = Word2Vec_set(mix_word,test_words)
    for i in range(m): # 循环测试样本中的m个样本
        
        for k in range(K):
            
            label_index = np.where(classVec==list_set_classVec[k])[0]
            label_data = Vec_words[label_index,:] 
            same_words = np.where((vec_test_words[i] + label_data)>1)[0] #Vec_words中每一句与test_words相同的词个数
            molecule = same_words.shape[0] + gamma # 分子部分
            denominator = (classVec == list_set_classVec[k]).sum() + len(mix_word) * gamma
            
            # 后验概率
            Pos_proba[i,k] = molecule / denominator
        
        
    argmax_index = np.argmax(Pos_proba,axis=1)
    
    if is_print:
        
        print(list_set_classVec)
        print(Pos_proba)
    
    return argmax_index

选用一段样本测试

In [57]:
test_words = [['my','dog','has','flea','problems','help','please'],['you','are','stupid'],['asshole']]

predict_label = NaiveByes_word(mix_word,Vec_words,test_words,classVec,1,True)
print("predict label: ",predict_label)

the word you is not in my Vocabulary!
the word are is not in my Vocabulary!
the word asshole is not in my Vocabulary!
[0, 1]
[[0.28571429 0.08571429]
 [0.02857143 0.11428571]
 [0.02857143 0.02857143]]
predict label:  [0 1 0]


可以看出,如果某个单词不在词集中,那么即使是恶意词汇也是不能区分的,所以Naive Bayes是很受训练样本的控制的.如果训练样本过小,那么很肯能会出现很多分类错误的情况.

# Summary
- Naive Bayes 先计算先验概率,再计算条件概率,最后得到后验概率.
- Multinomial Naive Bayes受训练样本的影响非常大,如果训练样本过小,那么测试结果会很差.可以体现为,没有学习的过程,只是将测试样本(传入的样本)与现有的样本之间进行概率对比,"相似度高"则分类为该类.

- 在Multinomial Naive Bayes也没有涉及到学习最优参数,其实对于KNN,Decision Tree,都没有在学习参数.这会使得一些连续型的数据没办法分类.

除了Multinomial Naive Bayes之外实际上还有很多衍生版本,这些衍生版本有些会涉及学习最优参数比如Gaussian Naive Bayes,等等,详情见[scikit-learn](https://scikit-learn.org/stable/modules/naive_bayes.html)

# Homework

学习Gaussian Naive Bayes,并手动完成鸢尾花数据集的分类并使用scikit中的Gaussian Naive Bayes检测结果

```python
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, [0, 1, -1]])
    return data[:,:2], data[:,-1]

X, y = create_data()```